In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
import unicodedata
from pathlib import Path
from typing import Dict, List, Optional
import time

class IndexadorChatCUCEI:
    def __init__(self, persist_dir="./chroma_db"):
        """
        Inicializa el sistema de indexación
        
        Args:
            persist_dir: Directorio donde se guardará ChromaDB
        """
        print("="*70)
        print("INDEXADOR ChatCUCEI - Sistema RAG Optimizado")
        print("="*70)
        
        
        print("\nCargando modelo de embeddings...")
        self.embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        print(" Modelo cargado: paraphrase-multilingual-MiniLM-L12-v2")
        
        
        print(f"Inicializando ChromaDB en: {persist_dir}")
        self.client = chromadb.PersistentClient(path=persist_dir)
        
        
        try:
            self.collection = self.client.get_collection("profesores_cucei")
            print(f"✅ Colección existente: {self.collection.count()} documentos")
        except:
            self.collection = self.client.create_collection(
                name="profesores_cucei",
                metadata={"hnsw:space": "cosine"}
            )
            print("✅ Nueva colección creada")
        
        print()
    
    @staticmethod
    def normalizar_nombre(nombre: str) -> str:
        
        
        nombre = nombre.replace('"', '').replace("'", '')
        
        
        nombre = nombre.replace(',', ' ')
        nombre = ' '.join(nombre.split())  
        
        
        nombre = nombre.upper()
        
       
        nombre = unicodedata.normalize('NFKD', nombre)
        nombre = ''.join([c for c in nombre if not unicodedata.combining(c)])
        
        return nombre.strip()
    
    def limpiar_comentario(self, comentario: str) -> Optional[str]:
       
        if pd.isna(comentario) or not comentario:
            return None
        
        comentario = str(comentario).strip()
        
        
        comentario = re.sub(r'^Calificación:\s*\d+\.?\d*/10\s*-?\s*', '', comentario)
        
        
        if comentario in ['(No hay información del maestro)', '', 'N/A', 'nan']:
            return None
        
        if len(comentario) < 10:  
            return None
        
        return comentario.strip()
    
    def extraer_calificacion(self, texto: str) -> float:
        
        match = re.search(r'Calificación:\s*(\d+\.?\d*)/10', texto)
        return float(match.group(1)) if match else 0.0
    
    def extraer_tags(self, texto: str, max_tags: int = 5) -> List[str]:
       
        if " - " not in texto:
            return []
        
        
        tags_text = texto.split(" - ", 1)[1]
        
       
        tags = []
        for tag in tags_text.split(","):
            tag = tag.strip().upper()
            if tag and len(tag) > 2:  
                tags.append(tag)
        
        return tags[:max_tags]
    
    def crear_documento_embedding(self, row: pd.Series, comentario_limpio: str) -> str:
       
        profesor = row['PROFESOR']
        materia = row['MATERIA'] if pd.notna(row['MATERIA']) else ""
        depto = row['DEPARTAMENTO'] if pd.notna(row['DEPARTAMENTO']) else ""
        calificacion = self.extraer_calificacion(row['COMENTARIOS'])
        tags = self.extraer_tags(row['COMENTARIOS'])
        
       
        documento = f"""
        Profesor: {profesor}
        Nombre del maestro: {profesor}
        Evaluación del profesor {profesor}
        Materia: {materia}
        Departamento: {depto}
        Calificación: {calificacion}/10
        Características: {', '.join(tags) if tags else 'N/A'}
        Comentarios: {comentario_limpio}
        """.strip()
        
        return documento
    
    def indexar_csv(self, csv_path: str, batch_size: int = 100):
      
        print(f"{'='*70}")
        print("INICIANDO INDEXACIÓN")
        print(f"{'='*70}\n")
        
       
        csv_path = Path(csv_path)
        if not csv_path.exists():
            raise FileNotFoundError(f"No se encuentra: {csv_path}")
        
        print(f" Archivo: {csv_path.name}")
        
        
        print(" Leyendo CSV...")
        df = pd.read_csv(csv_path, encoding='utf-8')
        print(f"   Total filas: {len(df):,}")
        
        
        documentos = []
        metadatas = []
        ids = []
        
        stats = {
            'total': len(df),
            'procesados': 0,
            'omitidos': 0,
            'sin_comentarios': 0,
            'sin_profesor': 0
        }
        
        print("\n Procesando datos...")
        inicio = time.time()
        
        for idx, row in df.iterrows():
            
            if pd.isna(row['PROFESOR']) or not str(row['PROFESOR']).strip():
                stats['sin_profesor'] += 1
                stats['omitidos'] += 1
                continue
            
            profesor = str(row['PROFESOR']).strip()
            profesor_normalizado = self.normalizar_nombre(profesor)
            
            
            comentario_limpio = self.limpiar_comentario(row['COMENTARIOS'])
            if not comentario_limpio:
                stats['sin_comentarios'] += 1
                stats['omitidos'] += 1
                continue
            
            
            comentarios_original = str(row['COMENTARIOS'])
            calificacion = self.extraer_calificacion(comentarios_original)
            tags = self.extraer_tags(comentarios_original)
            
            materia = str(row['MATERIA']) if pd.notna(row['MATERIA']) else ""
            depto = str(row['DEPARTAMENTO']) if pd.notna(row['DEPARTAMENTO']) else ""
            division = str(row['DIVISION']) if 'DIVISION' in row and pd.notna(row['DIVISION']) else ""
            
            
            documento = self.crear_documento_embedding(row, comentario_limpio)
            documentos.append(documento)
            
            
            metadatas.append({
                "profesor": profesor,
                "profesor_normalizado": profesor_normalizado,
                "materia": materia,
                "departamento": depto,
                "division": division,
                "calificacion": float(calificacion),
                "tags": ", ".join(tags) if tags else "",
                "comentarios": comentario_limpio,
                "comentarios_original": comentarios_original
            })
            
            ids.append(f"resena_{idx}")
            stats['procesados'] += 1
            
            
            if (idx + 1) % 500 == 0:
                print(f"   Procesados: {idx + 1:,}/{len(df):,}")
        
        print(f"\n Procesamiento completado en {time.time() - inicio:.2f}s")
        print(f"\n Resumen:")
        print(f"   Total filas: {stats['total']:,}")
        print(f"   Procesados: {stats['procesados']:,}")
        print(f"   Omitidos: {stats['omitidos']:,}")
        print(f"     - Sin profesor: {stats['sin_profesor']:,}")
        print(f"     - Sin comentarios válidos: {stats['sin_comentarios']:,}")
        
        
        print(f"\n Generando embeddings e indexando en ChromaDB...")
        print(f"   Batch size: {batch_size}")
        
        inicio = time.time()
        
        for i in range(0, len(documentos), batch_size):
            batch_docs = documentos[i:i+batch_size]
            batch_metas = metadatas[i:i+batch_size]
            batch_ids = ids[i:i+batch_size]
            
            self.collection.add(
                documents=batch_docs,
                metadatas=batch_metas,
                ids=batch_ids
            )
            
            if (i + batch_size) % 500 == 0 or i + batch_size >= len(documentos):
                current = min(i + batch_size, len(documentos))
                print(f"   ✓ {current:,}/{len(documentos):,} indexados")
        
        tiempo_indexacion = time.time() - inicio
        
        print(f"\n Indexación completada en {tiempo_indexacion:.2f}s")
        print(f" Total documentos en ChromaDB: {self.collection.count():,}")
        print(f" Velocidad: {len(documentos)/tiempo_indexacion:.1f} docs/segundo")
    
    def verificar_indexacion(self, n_samples: int = 5):
        
        print(f"\n{'='*70}")
        print("VERIFICACIÓN DE INDEXACIÓN")
        print(f"{'='*70}\n")
        
        total = self.collection.count()
        print(f" Total documentos: {total:,}\n")
        
        if total == 0:
            print("  Base de datos vacía")
            return
        
        
        peek = self.collection.peek(limit=n_samples)
        
        print(f" Muestra de {n_samples} documentos:\n")
        
        for i, (doc, meta) in enumerate(zip(peek['documents'], peek['metadatas']), 1):
            print(f"{'─'*70}")
            print(f"Documento #{i}")
            print(f"{'─'*70}")
            print(f"Profesor: {meta['profesor']}")
            print(f"Normalizado: {meta['profesor_normalizado']}")
            print(f"Calificación: {meta['calificacion']}/10")
            if meta['tags']:
                tags_preview = ', '.join(meta['tags'].split(',')[:3])
                print(f"Tags: {tags_preview}")
            if meta['materia']:
                print(f"Materia: {meta['materia']}")
            print(f"Comentario: {meta['comentarios'][:100]}...")
            print()
        
        
        print(f"{'─'*70}")
        print(" Estadísticas generales:")
        print(f"   Total profesores únicos: ~{total//5} (estimado)")
        print(f"   Promedio reseñas/profesor: ~5")
        print(f"{'─'*70}\n")
    
    def test_busqueda(self, queries: List[str], n_results: int = 3):
       
        print(f"{'='*70}")
        print("PRUEBA DE BÚSQUEDA")
        print(f"{'='*70}\n")
        
        for i, query in enumerate(queries, 1):
            print(f"{'─'*70}")
            print(f"Query #{i}: {query}")
            print(f"{'─'*70}")
            
            results = self.collection.query(
                query_texts=[query],
                n_results=n_results
            )
            
            if not results['metadatas'][0]:
                print("⚠️  Sin resultados\n")
                continue
            
            for j, meta in enumerate(results['metadatas'][0], 1):
                print(f"\nResultado {j}:")
                print(f"  Profesor: {meta['profesor']}")
                print(f"  Calificación: {meta['calificacion']}/10")
                if meta['tags']:
                    print(f"  Tags: {meta['tags'][:60]}...")
                print(f"  Comentario: {meta['comentarios'][:120]}...")
            
            print()
        
        print(f"{'='*70}\n")
    
    def reiniciar(self):
        
        print("\n  REINICIANDO BASE DE DATOS...")
        try:
            self.client.delete_collection("profesores_cucei")
            print(" Colección borrada")
        except:
            print(" No había colección previa")
        
        self.collection = self.client.create_collection(
            name="profesores_cucei",
            metadata={"hnsw:space": "cosine"}
        )
        print(" Nueva colección creada\n")


def main():

    CSV_PATH = r"# === NOTE: Replace with local path ==="
    CHROMA_DB_PATH = "./chroma_db"
  
    indexador = IndexadorChatCUCEI(persist_dir=CHROMA_DB_PATH)
    
    
    print(f"{'='*70}")
    print("MENÚ DE INDEXACIÓN")
    print(f"{'='*70}\n")
    
    documentos_existentes = indexador.collection.count()
    
    if documentos_existentes > 0:
        print(f" Base de datos existente: {documentos_existentes:,} documentos\n")
        print("Opciones:")
        print("  1. Usar datos existentes (verificar)")
        print("  2. Re-indexar desde cero (BORRA datos existentes)")
        print("  3. Ver estadísticas")
        print("  4. Probar búsquedas")
        print("  5. Salir")
        
        opcion = input("\nSelecciona (1-5): ").strip()
        
        if opcion == "1":
            indexador.verificar_indexacion(n_samples=10)
            
        elif opcion == "2":
            confirmacion = input("\n  Esto borrará todos los datos. ¿Continuar? (si/no): ")
            if confirmacion.lower() in ['si', 's', 'yes', 'y']:
                indexador.reiniciar()
                indexador.indexar_csv(CSV_PATH)
                indexador.verificar_indexacion()
            else:
                print(" Operación cancelada")
                
        elif opcion == "3":
            indexador.verificar_indexacion(n_samples=10)
            
        elif opcion == "4":
            test_queries = [
                "¿Qué opinas de ABEL MARQUEZ SOTO?",
                "¿Cómo es ELIZABETH ACEVES FERNANDEZ?",
                "Recomiendas a ADRIAN CERVANTES LOMELI?",
                "Info sobre MIGUEL ABUNDIS SANCHEZ"
            ]
            indexador.test_busqueda(test_queries)
            
        else:
            print(" Saliendo...")
            return
    
    else:
        print("  Base de datos vacía\n")
        print("Iniciando indexación completa...")
        indexador.indexar_csv(CSV_PATH)
        indexador.verificar_indexacion()
        
       
        print("\n¿Ejecutar pruebas de búsqueda? (s/n): ", end="")
        if input().lower() in ['s', 'si', 'y', 'yes']:
            test_queries = [
                "¿Qué opinas de ABEL MARQUEZ SOTO?",
                "¿Cómo es ELIZABETH ACEVES FERNANDEZ?",
                "Recomiendas a ADRIAN CERVANTES?"
            ]
            indexador.test_busqueda(test_queries)
    
    # Mensaje final
    print(f"{'='*70}")
    print(" INDEXACIÓN COMPLETADA")
    print(f"{'='*70}")
    print(f"\n Base de datos lista en: {CHROMA_DB_PATH}")
    print(f" Total documentos: {indexador.collection.count():,}")
    print(f"\n SIGUIENTE PASO:")
    print(f"   Ejecuta tu script de fine-tuning:")
    print(f"   python train_phi3_hybrid.py")
    print(f"\n   O prueba inferencia directa:")
    print(f"   python inference_hybrid.py\n")


if __name__ == "__main__":
    main()